<a href="https://colab.research.google.com/github/PavelTitov2993/Github_projects/blob/main/materials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Section with the import of the libraries and reading input files.
from sympy import *
import pandas as pd

lst = ['Number','A','B','C','D','BC','2']
points_dat = pd.read_csv('stg_0_2607.cel', sep='\s+', names=lst, engine='python')
points_dat.drop(columns=['2'], inplace=True)

lst = ['Number','X','Y','11']
coord_dat = pd.read_csv('stg_0_2607.vrt', sep='\s+', names=lst, engine='python')
coord_dat.drop(columns=['11'], inplace=True)
print(points_dat.head(5))
print(coord_dat.head(5))


# Here we define the function with the main goal of calculating the coordinates 
# of every cell's center.

def find_cell_center(x1, y1, x2, y2, x3, y3, x4, y4):
    a1, b1, c1, d1 = Point(x1, y1), Point(x2, y2), Point(x3, y3), Point(x4, y4)
    t1 = Triangle(a1, b1, c1)
    t2 = Triangle(c1, d1, a1)
    l1 = intersection(t1.medians[a1], t1.medians[b1], t1.medians[c1])
    l2 = intersection(t2.medians[c1], t2.medians[d1], t2.medians[a1])
    pair_1 = Line(l1[0], l2[0])
    t1 = Triangle(b1, c1, d1)
    t2 = Triangle(d1, a1, b1)
    l1 = intersection(t1.medians[b1], t1.medians[c1], t1.medians[d1])
    l2 = intersection(t2.medians[d1], t2.medians[a1], t2.medians[b1])
    pair_2 = Line(l1[0], l2[0])
    res = intersection(pair_1, pair_2)
    return float(res[0][0]), float(res[0][1])


# Next, we append 2 columns - x and y coordinates of the center - to the 
# dataframe with information about cells.

X_C = []
Y_C = []
a_nodes = points_dat.A
b_nodes = points_dat.B
c_nodes = points_dat.C
d_nodes = points_dat.D
for i in range(len(points_dat.index)):
    x_a = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == a_nodes[i]]['X'])
    y_a = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == a_nodes[i]]['Y'])
    x_b = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == b_nodes[i]]['X'])
    y_b = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == b_nodes[i]]['Y'])
    x_c = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == c_nodes[i]]['X'])
    y_c = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == c_nodes[i]]['Y'])
    x_d = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == d_nodes[i]]['X'])
    y_d = float(coord_dat.loc[coord_dat.loc[:, 'Number'] == d_nodes[i]]['Y'])
    x_centr, y_centr = find_cell_center(x_a, y_a, x_b, y_b, x_c, y_c, x_d, y_d)
    X_C.append(x_centr)
    Y_C.append(y_centr)


X_Cr = pd.Series(X_C, name = 'X_center')
Y_Cr = pd.Series(Y_C, name = 'Y_center')
points_dat.insert(6, 'X_center', X_Cr)
points_dat.insert(7, 'Y_center', Y_Cr)

# This block of code is responsible for renumerating all of the cells. 
# Cells should be numerated from bottom to top, from left to right. It is 
# essential because of the specifics of the used software.

points = points_dat.B
B_X = []
B_Y = []
for i in range(len(points_dat.index)):
  B_X.append(coord_dat.loc[coord_dat['Number'] == points[i]].X.iloc[0])
  B_Y.append(coord_dat.loc[coord_dat['Number'] == points[i]].Y.iloc[0])
B_X = pd.Series(B_X, name = 'B_X')
B_Y = pd.Series(B_Y, name = 'B_Y')
new_points_dat = pd.concat([points_dat, B_Y, B_X,], axis = 1)
new_points_dat.sort_values(by=["B_Y", "B_X"], inplace=True, ignore_index=True)
new_points_dat['Number'] = new_points_dat.index + 1
new_points_dat.drop(columns=['B_X', 'B_Y'], inplace=True)
print(new_points_dat.head())


# Due to the model size, calculation and renumeration on previous stages take 
# some time, so we save our progress by writing the results.

new_points_dat.to_csv('new_points.txt', sep=' ', index=False, header=False)
coord_dat.to_csv('new_nodes.txt', sep=' ', index=False, header=False)

   Number     A     B     C     D  BC
0       1    13  1490  1799   376   3
1       2  1490  1491  1800  1799   3
2       3  1491  1492  1801  1800   3
3       4  1492  1493  1802  1801   3
4       5  1493    52  1803  1802   3
   Number             X       Y
0       1  2.322500e+00  3.4600
1       2  2.047500e+00  3.4600
2       3  1.685000e+00  2.8500
3       4 -5.583955e-10  1.7525
4       5  1.685000e+00  1.7525
   Number     A    B    C     D  BC  X_center  Y_center
0       1   201   30  744  6563   4  0.025281  0.015353
1       2  6563  744  745  6572   4  0.075818  0.015966
2       3  6572  745  746  6581   4  0.126344  0.017150
3       4  6581  746  747  6590   4  0.176855  0.018907
4       5  6590  747  748  6599   4  0.227342  0.021238


In [ ]:
# And here we read them again
from sympy import *
import pandas as pd
lst = ['Number','A','B','C','D','BC','X_center','Y_center']
new_points_dat = pd.read_csv('new_points.txt', sep='\s+', names=lst, engine='python')
print(new_points_dat.head())
lst = ['Number','X','Y']
coord_dat = pd.read_csv('new_nodes.txt', sep='\s+', names=lst, engine='python')
print(coord_dat.head())

   Number     A    B    C     D  BC  X_center  Y_center
0       1   201   30  744  6563   4  0.025281  0.015353
1       2  6563  744  745  6572   4  0.075818  0.015966
2       3  6572  745  746  6581   4  0.126344  0.017150
3       4  6581  746  747  6590   4  0.176855  0.018907
4       5  6590  747  748  6599   4  0.227342  0.021238
   Number             X       Y
0       1  2.322500e+00  3.4600
1       2  2.047500e+00  3.4600
2       3  1.685000e+00  2.8500
3       4 -5.583955e-10  1.7525
4       5  1.685000e+00  1.7525


In [ ]:
# Then we define two functions. It marks (assigns a number to) the chosen group 
# of cells depending on their coordinates and belonging to the one of the main 
# bigger areas defined in CAD software. The first one is used to build a layered 
# structure, and the second is used to draw more complex objects (so calles 
# glasses) defined with both x and y coordinates.
#

def change_material(cells, y_1, y_2, bc_old, bc_new):
    cells.loc[(cells['BC'] == bc_old) & (cells['Y_center'] > y_1) & (cells['Y_center'] < y_2), 'BC'] = bc_new
    
def glass(cells, x_1, x_2, y_1, y_2, bc_old, bc_new):
    cells.loc[(cells['BC'] == bc_old) & (cells['Y_center'] > y_1) & (cells['Y_center'] < y_2) & (cells['X_center'] > x_1) & (cells['X_center'] < x_2), 'BC'] = bc_new

# Here we make one test of the new function
change_material(new_points_dat, 1.345, 1.4925, 222, 12)  # layer 12

In [ ]:
# After we checked that the function change_material is working, we can apply 
# it to draw every picture we want. In our case we make layered structure 
# depicting melt stratification in the active zone of the reactor. Names given 
# at the end of every string don't have an important meaning, they were made 
# just for the convenience.


change_material(new_points_dat, 1.2325, 1.35, 222, 11)  # layer 11
change_material(new_points_dat, 1.1025, 1.2325, 222, 10)  # layer 10
change_material(new_points_dat, 0.95, 1.1025, 222, 9)  # layer 9
change_material(new_points_dat, 0.82, 0.95, 222, 8)  # layer 8
change_material(new_points_dat, 0.461, 0.82, 222, 7)  # layer 7

change_material(new_points_dat, 1.875, 2.85, 111, 46)  # gap_1 matr 46
change_material(new_points_dat, 1.6, 1.875, 111, 45)  # gap_2 matr 45
change_material(new_points_dat, 1.445, 1.6, 111, 44)  # gap_3 matr 44
change_material(new_points_dat, 1.345, 1.445, 111, 43)  # gap_4 matr 43
change_material(new_points_dat, 1.2325, 1.345, 111, 42)  # gap_5 matr 42
change_material(new_points_dat, 1.12, 1.2325, 111, 41)  # gap_6 matr 41
change_material(new_points_dat, 1.0, 1.12, 111, 40)  # gap_7 matr 40
change_material(new_points_dat, 0.905, 1.0, 111, 39)  # gap_8 matr 39
change_material(new_points_dat, 0.8, 0.905, 111, 38)  # gap_9 matr 38
change_material(new_points_dat, 0.656, 0.8, 111, 37)  # gap_10 matr 37
change_material(new_points_dat, 0.5505, 0.656, 111, 36)  # gap_11 matr 36
change_material(new_points_dat, 0.445, 0.5505, 111, 35)  # gap_12 matr 35
change_material(new_points_dat, 0.3395, 0.445, 111, 34)  # gap_13 matr 34
change_material(new_points_dat, 0.234, 0.3395, 111, 33)  # gap_14 matr 33

glass(new_points_dat, 0.11, 0.2, 0.461, 0.82, 7, 5)  # glass 1
glass(new_points_dat, 0.11, 0.2, 0, 5, 8, 5)
glass(new_points_dat, 0.11, 0.2, 0.95, 1.1025, 9, 5)
glass(new_points_dat, 0.11, 0.2, 0, 1.194, 10, 5)

glass(new_points_dat, 0.06, 0.232, 0, 5, 10, 6)
glass(new_points_dat, 0.06, 0.232, 0, 5, 11, 6)
glass(new_points_dat, 0.06, 0.232, 0, 5, 12, 6)
glass(new_points_dat, 0.06, 0.232, 0, 1.65, 13, 6)
glass(new_points_dat, 0.06, 0.232, 0, 1.72, 14, 6)

glass(new_points_dat, 0.44, 0.52, 0.461, 0.82, 7, 5)  # glass 2
glass(new_points_dat, 0.44, 0.52, 0.82, 0.95, 8, 5)
glass(new_points_dat, 0.44, 0.52, 0.95, 1.1025, 9, 5)
glass(new_points_dat, 0.44, 0.52, 0, 1.194, 10, 5)

glass(new_points_dat, 0.44, 0.59, 1.194, 1.5, 10, 6)
glass(new_points_dat, 0.44, 0.59, 0, 5, 11, 6)
glass(new_points_dat, 0.44, 0.59, 0, 5, 12, 6)
glass(new_points_dat, 0.44, 0.59, 0, 1.65, 13, 6)
glass(new_points_dat, 0.44, 0.59, 0, 1.72, 14, 6)

glass(new_points_dat, 0.33, 0.44, 1.194, 1.5, 10, 5)
glass(new_points_dat, 0.33, 0.44, 0, 5, 11, 5)
glass(new_points_dat, 0.33, 0.44, 0, 5, 12, 5)
glass(new_points_dat, 0.33, 0.44, 0, 1.65, 13, 5)
glass(new_points_dat, 0.33, 0.44, 0, 1.72, 14, 5)

glass(new_points_dat, 0.725, 0.81, 0.461, 0.82, 7, 5)  # glass 3
glass(new_points_dat, 0.725, 0.81, 0.82, 0.95, 8, 5)
glass(new_points_dat, 0.725, 0.81, 0.95, 1.1025, 9, 5)
glass(new_points_dat, 0.725, 0.81, 0, 1.194, 10, 5)

glass(new_points_dat, 0.73, 0.88, 1.194, 1.5, 10, 6)
glass(new_points_dat, 0.73, 0.88, 0, 5, 11, 6)
glass(new_points_dat, 0.73, 0.88, 0, 5, 12, 6)
glass(new_points_dat, 0.73, 0.88, 0, 1.65, 13, 6)
glass(new_points_dat, 0.73, 0.88, 0, 1.72, 14, 6)

glass(new_points_dat, 0.65, 0.73, 1.194, 1.5, 10, 5)
glass(new_points_dat, 0.65, 0.73, 0, 5, 11, 5)
glass(new_points_dat, 0.65, 0.73, 0, 5, 12, 5)
glass(new_points_dat, 0.65, 0.73, 0, 1.65, 13, 5)
glass(new_points_dat, 0.65, 0.73, 0, 1.72, 14, 5)

glass(new_points_dat, 1.02, 1.1, 0.461, 0.82, 7, 5)  # glass 4
glass(new_points_dat, 1.02, 1.1, 0.82, 0.95, 8, 5)
glass(new_points_dat, 1.02, 1.1, 0.95, 1.1025, 9, 5)
glass(new_points_dat, 1.02, 1.1, 0, 1.194, 10, 5)

glass(new_points_dat, 1.02, 1.17, 1.194, 1.5, 10, 6)
glass(new_points_dat, 1.02, 1.17, 0, 5, 11, 6)
glass(new_points_dat, 1.02, 1.17, 0, 5, 12, 6)
glass(new_points_dat, 1.02, 1.17, 0, 1.65, 13, 6)
glass(new_points_dat, 1.02, 1.17, 0, 1.72, 14, 6)

glass(new_points_dat, 0.94, 1.02, 1.194, 1.5, 10, 5)
glass(new_points_dat, 0.94, 1.02, 0, 5, 11, 5)
glass(new_points_dat, 0.94, 1.02, 0, 5, 12, 5)
glass(new_points_dat, 0.94, 1.02, 0, 1.65, 13, 5)
glass(new_points_dat, 0.94, 1.02, 0, 1.72, 14, 5)

glass(new_points_dat, 1.28, 1.38, 0.461, 0.82, 7, 5)  # glass 5
glass(new_points_dat, 1.28, 1.38, 0.82, 0.95, 8, 5)
glass(new_points_dat, 1.28, 1.38, 0.95, 1.1025, 9, 5)
glass(new_points_dat, 1.28, 1.38, 0, 1.194, 10, 5)

glass(new_points_dat, 1.25, 1.33, 1.194, 1.5, 10, 5)
glass(new_points_dat, 1.25, 1.33, 0, 5, 11, 5)
glass(new_points_dat, 1.25, 1.33, 0, 5, 12, 5)
glass(new_points_dat, 1.25, 1.33, 0, 1.65, 13, 5)
glass(new_points_dat, 1.25, 1.33, 0, 1.72, 14, 5)

glass(new_points_dat, 1.33, 1.58, 1.194, 1.5, 10, 6)
glass(new_points_dat, 1.33, 1.58, 0, 5, 11, 6)
glass(new_points_dat, 1.33, 1.58, 0, 5, 12, 6)
glass(new_points_dat, 1.33, 1.58, 0, 1.65, 13, 6)
glass(new_points_dat, 1.33, 1.58, 0, 1.72, 14, 6)

# These two prints are needed to make sure that all of the cells now have their
# own "mark" and nothing was missed

print(new_points_dat.loc[new_points_dat.loc[:, 'BC'] == 222])
print(new_points_dat.loc[new_points_dat.loc[:, 'BC'] == 111])

Empty DataFrame
Columns: [Number, A, B, C, D, BC, X_center, Y_center]
Index: []
Empty DataFrame
Columns: [Number, A, B, C, D, BC, X_center, Y_center]
Index: []


In [ ]:
# After we design everything we needed, next stage is to transform model data 
# to the specific format required by final software. So this section is 
# responsible for reformatting.

new_points_dat.drop(columns=['X_center', 'Y_center'], inplace=True)
coord_dat.insert(1, 'zero', '.0')

list_string_x = []
for num in coord_dat.loc[:, 'X']:
  elem = f"{num:9.7f}"
  if elem.startswith('0'):
    elem = elem[1:]
  elif elem.startswith('-0'):
    elem = elem[2:]
  list_string_x.append(elem)

list_string_y = []
for num in coord_dat.loc[:, 'Y']:
  elem = f"{num:9.7f}"
  if elem.startswith('0'):
    elem = elem[1:]
  elif elem.startswith('-0'):
    elem = elem[2:]
  list_string_y.append(elem)

coord_dat.drop(columns=['X', 'Y'], inplace=True)
coord_dat.insert(2, 'X', list_string_x)
coord_dat.insert(3, 'Y', list_string_y)

# This output is for visual inspection of the data.

print(new_points_dat.head(10))
print(coord_dat.head(10))

# And the last thing is to write our data to the files. The final format is
# pretty demanding, data should be written in only one specific way.

output_cells = open('e0lemn', 'w')
for i in range(len(new_points_dat)):
  output_cells.write(f"{new_points_dat.loc[i, 'Number']: >5}" + f"{new_points_dat.loc[i, 'A']: >5}" + f"{new_points_dat.loc[i, 'B']: >5}" + f"{new_points_dat.loc[i, 'C']: >5}" + f"{new_points_dat.loc[i, 'D']: >5}" + f"{new_points_dat.loc[i, 'BC']: >5}" + '\n')
output_cells.close()

output_nodes = open('n0odc', 'w')
for i in range(len(coord_dat)):
  output_nodes.write(f"{coord_dat.loc[i, 'Number']: >5}" + f"{coord_dat.loc[i, 'zero']: >5}" + f"{coord_dat.loc[i, 'X']: >10}" + f"{coord_dat.loc[i, 'Y']: >10}" + '\n')
output_nodes.close()

   Number             X         Y
0       1  2.322500e+00  3.460000
1       2  2.047500e+00  3.460000
2       3  1.685000e+00  2.850000
3       4 -5.583955e-10  1.752500
4       5  1.685000e+00  1.752500
5       6 -1.032546e-09  2.850000
6       7 -3.300000e-16  0.340000
7       8 -1.800000e-16  0.234000
8       9  1.314828e+00  0.459262
9      10  1.171124e+00  0.602966
   Number     A    B    C     D  BC
0       1   201   30  744  6563   4
1       2  6563  744  745  6572   4
2       3  6572  745  746  6581   4
3       4  6581  746  747  6590   4
4       5  6590  747  748  6599   4
5       6  6599  748  749  6608   4
6       7  6608  749  750  6617   4
7       8  6617  750  751  6626   4
8       9  6626  751  752  6635   4
9      10  6635  752  753  6644   4
   Number zero          X          Y
0       1   .0  2.3225000  3.4600000
1       2   .0  2.0475000  3.4600000
2       3   .0  1.6850000  2.8500000
3       4   .0   .0000000  1.7525000
4       5   .0  1.6850000  1.7525000
5       